In [1]:
import os
import re
import time
import yt_dlp
import requests
import argparse
import subprocess
import multiprocessing

In [2]:
JS_FETCHER_BASE_URL = "https://storage.googleapis.com/data.yt8m.org/2/j/v/"
VIDEO_ID_FETCHER_BASE_URL = "https://storage.googleapis.com/data.yt8m.org/2/j/i/"
METADATA_CSV_FILE = "vocabulary.csv"

In [3]:
def make_get_request(url):
    try:
        resp = requests.get(url)
        resp.raise_for_status()
    except Exception as err:
        return True, err
    else:
        return False, resp.text

In [4]:
def get_knowledge_graph_id(category_names):
    id_mapping_dict = {}
    with open(METADATA_CSV_FILE, "r") as mtd:
        kg_data = mtd.read()

    kg_data = kg_data.split("\n")
    kg_data = [i.split(",") for i in kg_data]
    set_categories = set(category_names)

    for k in kg_data:
        if k[-1] in set_categories:
            id_mapping_dict[k[-1]] = k[-2]

    return id_mapping_dict

In [5]:
def download_video_using_youtube_dl(video_id, output_path):
    url = "http://youtube.com/v/"+str(video_id)
    ydl_opts = {
        # Specify the video format as MP4 and also download the best audio available.
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
        # Output template for saving the video.
        'outtmpl': output_path + "/" + video_id + '.%(ext)s',
        # Specify ffmpeg location if it's not in PATH.
        'ffmpeg_location': './ffmpeg-git-20240301-amd64-static/ffmpeg',
        # Define postprocessors to extract audio and save it as WAV.
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            error_code = ydl.download([url])          
        return True, ""
    except:
        return False, ""

In [6]:
def run(selected_categories, output_dir, number_of_videos=500):
    id_mapping_dict = get_knowledge_graph_id(selected_categories)

    for key, value in id_mapping_dict.items():
        print("Fetching videos for '{}' category".format(key))
        err1, js_data = make_get_request(JS_FETCHER_BASE_URL+value.split("/")[-1]+".js")
        if err1:
            print("Error: {}, with retrieving JS data for category '{}', skipping".format(str(js_data),key))
            return
        js_data = eval(js_data.lstrip('p("'+value.split("/")[-1]+'",').rstrip(');'))
        tf_records_ids = js_data[2:]
        tf_records_ids = tf_records_ids[:number_of_videos] # keep more ids
        tf_records_ids_first_two_chars = [i[:2] for i in tf_records_ids]

        video_ids_list = []
        for ids, two_char_ids in zip(tf_records_ids, tf_records_ids_first_two_chars):
            url = VIDEO_ID_FETCHER_BASE_URL + two_char_ids + "/" + ids + ".js"
            err2, id_data = make_get_request(url)
            if err2:
                continue
            video_id = re.findall(r"[a-zA-Z0-9_-]{11}", id_data)[0]
            video_ids_list.append(video_id)

        limit = 0

        for vid in video_ids_list:
            err, _ = download_video_using_youtube_dl(vid, output_dir)
            if not err:
                limit += 1
                print("Video id: {} downloaded successfully.".format(vid))
            else:
                print("Video id: {} download unsuccessful.".format(vid))
            if limit == number_of_videos:
                break

In [7]:
run(["Airplane"], "/data/ai_club/team_13_2023-24/videos")

Fetching videos for 'Airplane' category
[youtube] Extracting URL: http://youtube.com/v/Cn0w1tpt22Y
[youtube] Cn0w1tpt22Y: Downloading webpage
[youtube] Cn0w1tpt22Y: Downloading ios player API JSON
[youtube] Cn0w1tpt22Y: Downloading android player API JSON
[youtube] Cn0w1tpt22Y: Downloading m3u8 information
[info] Cn0w1tpt22Y: Downloading 1 format(s): 609+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 38
[download] Destination: /data/ai_club/team_13_2023-24/videos/Cn0w1tpt22Y.f609.mp4
[download] 100% of   17.38MiB in 00:00:01 at 10.71MiB/s                  
[download] Destination: /data/ai_club/team_13_2023-24/videos/Cn0w1tpt22Y.f140.m4a
[download] 100% of    2.95MiB in 00:00:00 at 18.89MiB/s  
[Merger] Merging formats into "/data/ai_club/team_13_2023-24/videos/Cn0w1tpt22Y.mp4"
Deleting original file /data/ai_club/team_13_2023-24/videos/Cn0w1tpt22Y.f609.mp4 (pass -k to keep)
Deleting original file /data/ai_club/team_13_2023-24/videos/Cn0w1tpt22Y.f140.m4a (pass -k

[youtube] 6zHaGkCGFhc: Downloading ios player API JSON
[youtube] 6zHaGkCGFhc: Downloading android player API JSON
[youtube] 6zHaGkCGFhc: Downloading m3u8 information
[info] 6zHaGkCGFhc: Downloading 1 format(s): 606+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 24
[download] Destination: /data/ai_club/team_13_2023-24/videos/6zHaGkCGFhc.f606.mp4
[download] 100% of    1.40MiB in 00:00:00 at 2.27MiB/s                   
[download] Destination: /data/ai_club/team_13_2023-24/videos/6zHaGkCGFhc.f140.m4a
[download] 100% of    2.10MiB in 00:00:00 at 37.01MiB/s  
[Merger] Merging formats into "/data/ai_club/team_13_2023-24/videos/6zHaGkCGFhc.mp4"
Deleting original file /data/ai_club/team_13_2023-24/videos/6zHaGkCGFhc.f140.m4a (pass -k to keep)
Deleting original file /data/ai_club/team_13_2023-24/videos/6zHaGkCGFhc.f606.mp4 (pass -k to keep)
Video id: 6zHaGkCGFhc download unsuccessful.
[youtube] Extracting URL: http://youtube.com/v/hhsjT2FSmxU
[youtube] hhsjT2FSmxU: Downl

[info] hupqmv-loSw: Downloading 1 format(s): 606+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 68
[download] Destination: /data/ai_club/team_13_2023-24/videos/hupqmv-loSw.f606.mp4
[download] 100% of   18.48MiB in 00:00:02 at 7.26MiB/s                   
[download] Destination: /data/ai_club/team_13_2023-24/videos/hupqmv-loSw.f140.m4a
[download] 100% of    5.49MiB in 00:00:00 at 40.28MiB/s  
[Merger] Merging formats into "/data/ai_club/team_13_2023-24/videos/hupqmv-loSw.mp4"
Deleting original file /data/ai_club/team_13_2023-24/videos/hupqmv-loSw.f140.m4a (pass -k to keep)
Deleting original file /data/ai_club/team_13_2023-24/videos/hupqmv-loSw.f606.mp4 (pass -k to keep)
[FixupStretched] Fixing aspect ratio of "/data/ai_club/team_13_2023-24/videos/hupqmv-loSw.mp4"
Video id: hupqmv-loSw download unsuccessful.
[youtube] Extracting URL: http://youtube.com/v/x5Wzci8pmg4
[youtube] x5Wzci8pmg4: Downloading webpage
[youtube] x5Wzci8pmg4: Downloading ios player API JSON
[

[download] 100% of   19.14MiB in 00:00:02 at 7.30MiB/s                   
[download] Destination: /data/ai_club/team_13_2023-24/videos/oide6b6K_ug.f140.m4a
[download] 100% of    3.21MiB in 00:00:00 at 20.72MiB/s  
[Merger] Merging formats into "/data/ai_club/team_13_2023-24/videos/oide6b6K_ug.mp4"
Deleting original file /data/ai_club/team_13_2023-24/videos/oide6b6K_ug.f231.mp4 (pass -k to keep)
Deleting original file /data/ai_club/team_13_2023-24/videos/oide6b6K_ug.f140.m4a (pass -k to keep)
Video id: oide6b6K_ug download unsuccessful.
[youtube] Extracting URL: http://youtube.com/v/4V1oTGfSzwU
[youtube] 4V1oTGfSzwU: Downloading webpage
[youtube] 4V1oTGfSzwU: Downloading ios player API JSON
[youtube] 4V1oTGfSzwU: Downloading android player API JSON
[youtube] 4V1oTGfSzwU: Downloading m3u8 information
[info] 4V1oTGfSzwU: Downloading 1 format(s): 135+140
[download] Destination: /data/ai_club/team_13_2023-24/videos/4V1oTGfSzwU.f135.mp4
[download] 100% of   15.73MiB in 00:00:00 at 47.99MiB/s

Deleting original file /data/ai_club/team_13_2023-24/videos/3EVZEhEwO_c.f140.m4a (pass -k to keep)
Video id: 3EVZEhEwO_c download unsuccessful.
[youtube] Extracting URL: http://youtube.com/v/FJ2j836N2C0
[youtube] FJ2j836N2C0: Downloading webpage
[youtube] FJ2j836N2C0: Downloading ios player API JSON
[youtube] FJ2j836N2C0: Downloading android player API JSON
[youtube] FJ2j836N2C0: Downloading m3u8 information
[info] FJ2j836N2C0: Downloading 1 format(s): 136+140
[download] Destination: /data/ai_club/team_13_2023-24/videos/FJ2j836N2C0.f136.mp4
[download] 100% of   55.38MiB in 00:00:01 at 44.46MiB/s    
[download] Destination: /data/ai_club/team_13_2023-24/videos/FJ2j836N2C0.f140.m4a
[download] 100% of    6.96MiB in 00:00:00 at 49.44MiB/s  
[Merger] Merging formats into "/data/ai_club/team_13_2023-24/videos/FJ2j836N2C0.mp4"
Deleting original file /data/ai_club/team_13_2023-24/videos/FJ2j836N2C0.f136.mp4 (pass -k to keep)
Deleting original file /data/ai_club/team_13_2023-24/videos/FJ2j836N2

...

ERROR: [youtube] tWND1DsojUY: Private video. Sign in if you've been granted access to this video


Video id: tWND1DsojUY downloaded successfully.
[youtube] Extracting URL: http://youtube.com/v/K2J-A0QJaMY
[youtube] K2J-A0QJaMY: Downloading webpage
[youtube] K2J-A0QJaMY: Downloading ios player API JSON
[youtube] K2J-A0QJaMY: Downloading android player API JSON
[youtube] K2J-A0QJaMY: Downloading m3u8 information
[info] K2J-A0QJaMY: Downloading 1 format(s): 617+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 45
[download] Destination: /data/ai_club/team_13_2023-24/videos/K2J-A0QJaMY.f617.mp4
[download] 100% of   25.79MiB in 00:00:20 at 1.25MiB/s                  
[download] Destination: /data/ai_club/team_13_2023-24/videos/K2J-A0QJaMY.f140.m4a
[download] 100% of    2.61MiB in 00:00:00 at 5.97MiB/s   
[Merger] Merging formats into "/data/ai_club/team_13_2023-24/videos/K2J-A0QJaMY.mp4"
Deleting original file /data/ai_club/team_13_2023-24/videos/K2J-A0QJaMY.f617.mp4 (pass -k to keep)
Deleting original file /data/ai_club/team_13_2023-24/videos/K2J-A0QJaMY.f140.m4a (p

ERROR: [youtube] GuCBxhuzEUA: Private video. Sign in if you've been granted access to this video


Video id: GuCBxhuzEUA downloaded successfully.
[youtube] Extracting URL: http://youtube.com/v/sqOrkmfHXGU
[youtube] sqOrkmfHXGU: Downloading webpage
[youtube] sqOrkmfHXGU: Downloading ios player API JSON
[youtube] sqOrkmfHXGU: Downloading android player API JSON
[youtube] sqOrkmfHXGU: Downloading m3u8 information
[info] sqOrkmfHXGU: Downloading 1 format(s): 606+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 46
[download] Destination: /data/ai_club/team_13_2023-24/videos/sqOrkmfHXGU.f606.mp4
[download] 100% of   10.47MiB in 00:00:18 at 580.68KiB/s                 
[download] Destination: /data/ai_club/team_13_2023-24/videos/sqOrkmfHXGU.f140.m4a
[download] 100% of    3.58MiB in 00:00:00 at 9.44MiB/s   
[Merger] Merging formats into "/data/ai_club/team_13_2023-24/videos/sqOrkmfHXGU.mp4"
Deleting original file /data/ai_club/team_13_2023-24/videos/sqOrkmfHXGU.f140.m4a (pass -k to keep)
Deleting original file /data/ai_club/team_13_2023-24/videos/sqOrkmfHXGU.f606.mp4 (

[youtube] ZDvGavyBpKI: Downloading m3u8 information
[info] ZDvGavyBpKI: Downloading 1 format(s): 614+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 34
[download] Destination: /data/ai_club/team_13_2023-24/videos/ZDvGavyBpKI.f614.mp4
[download] 100% of   30.29MiB in 00:00:16 at 1.86MiB/s                  
[download] Destination: /data/ai_club/team_13_2023-24/videos/ZDvGavyBpKI.f140.m4a
[download] 100% of    2.79MiB in 00:00:00 at 7.26MiB/s   
[Merger] Merging formats into "/data/ai_club/team_13_2023-24/videos/ZDvGavyBpKI.mp4"
Deleting original file /data/ai_club/team_13_2023-24/videos/ZDvGavyBpKI.f614.mp4 (pass -k to keep)
Deleting original file /data/ai_club/team_13_2023-24/videos/ZDvGavyBpKI.f140.m4a (pass -k to keep)
Video id: ZDvGavyBpKI download unsuccessful.
[youtube] Extracting URL: http://youtube.com/v/6_YJXEtuQgY
[youtube] 6_YJXEtuQgY: Downloading webpage
[youtube] 6_YJXEtuQgY: Downloading ios player API JSON
[youtube] 6_YJXEtuQgY: Downloading android pl

In [1]:
import os
import zipfile

# Define the directory to be zipped and the output zip file name
directory_path = '/data/ai_club/team_13_2023-24/videos/train'
zip_file_path = './export.zip'

# Create a zip file
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            # Create a proper file path to be added to the zip
            file_path = os.path.join(root, file)
            # Add file to the zip file; arcname is for no directory structure in the zip
            zipf.write(file_path, arcname=os.path.relpath(file_path, directory_path))

zip_file_path

'./export.zip'